In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
sys.path.append('../')


In [2]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
from loglizer.models import PCA, IsolationForest, DecisionTree, LR
from loglizer import dataloader, preprocessing
import pickle
from tqdm import tqdm
from sklearn.ensemble import IsolationForest as iForest
from sklearn.metrics import precision_recall_fscore_support


def get_x_y(windows, content2tempalte):
    x = []
    y = []
    for window in tqdm(windows):
        template_list = []
        y_list = []
        for item in window:
            template = content2tempalte.get(item["Content"], "")
            template_list.append(template)
            y_list.append(item["Label"])
        x.append(template_list)
        y.append(1 if sum(y_list) > 0 else 0)
    return x, y 


In [4]:
dataname = "BGL"

config_info = {
    "BGL": {
        "structure_file": "../../data/BGL/BGL.log_structured.csv",
        "pkl_path": "../../proceeded_data/BGL/BGL_ws=60m_s=60m_0.8train",
    },
    "Thunderbird": {
        "structure_file": "../../data/Thunderbird/Thunderbird_10m.log_structured_drain.csv",
        "pkl_path": "../../proceeded_data/Thunderbird/Thunderbird_ws=60m_s=60m_0.8train",
    },
}

structure_file = config_info[dataname]["structure_file"]
pkl_path = config_info[dataname]["pkl_path"]

In [5]:
parsed_result = pd.read_csv(structure_file)
content2tempalte = dict(zip(parsed_result["Content"], parsed_result["EventTemplate"]))

In [6]:
parsed_result["EventTemplate"].nunique()

2579

In [5]:
with open(os.path.join(pkl_path, "train.pkl"), "rb") as fr:
    train_windows = pickle.load(fr)
with open(os.path.join(pkl_path, "test.pkl"), "rb") as fr:
    test_windows = pickle.load(fr)

train_x, train_y = get_x_y(train_windows, content2tempalte)
test_x, test_y = get_x_y(test_windows, content2tempalte)

100%|██████████| 105/105 [00:11<00:00,  9.12it/s]


In [6]:
import numpy as np
feature_extractor = preprocessing.FeatureExtractor()
train_feat = feature_extractor.fit_transform(np.array(train_x), term_weighting='tf-idf', normalization='zero-mean')

/var/folders/4f/tg3l22wd0vxckm1jcr7nhdcm0000gn/T/ipykernel_71645/3588340611.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_feat = feature_extractor.fit_transform(np.array(train_x), term_weighting='tf-idf', normalization='zero-mean')


====== Transformed train data summary ======
Train data shape: 416-by-6058



In [7]:
print("train shape: {}".format(train_feat.shape))

train shape: (416, 6058)


In [12]:
model_name = "dt"

if model_name.lower() == "if":
    model = iForest(n_estimators=100, max_features=1)
    model.fit(train_feat)

    pred_train = model.decision_function(train_feat)
    proba_train = (pred_train-pred_train.min()) / (pred_train.max() - pred_train.min())
    
    test_feat = feature_extractor.transform(np.array(test_x))
    pred_test = model.decision_function(test_feat)
    proba_test = (pred_test-pred_test.min()) / (pred_test.max() - pred_test.min())

elif model_name.lower() == "dt":
    model = DecisionTree()
    model.fit(train_feat, train_y)

    proba_train = model.predict_proba(train_feat)[:, 1]
    
    test_feat = feature_extractor.transform(np.array(test_x))
    proba_test = model.predict_proba(test_feat)[:, 1]
    
elif model_name.lower() == "lr":
    model = LR()
    model.fit(train_feat, train_y)
    proba_train = model.predict_proba(train_feat)[:, 1]

    test_feat = feature_extractor.transform(np.array(test_x))
    proba_test = model.predict_proba(test_feat)[:, 1]

====== Model summary ======
====== Transformed test data summary ======


/var/folders/4f/tg3l22wd0vxckm1jcr7nhdcm0000gn/T/ipykernel_71645/592350195.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_feat = feature_extractor.transform(np.array(test_x))
/Users/liujinyang/Codes/ScaleAD/baselines/loglizer/loglizer/preprocessing.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_df[event] = [0] * len(X_df)
/Users/liujinyang/Codes/ScaleAD/baselines/loglizer/loglizer/preprocessing.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

Test data shape: 105-by-6058



In [13]:
th = 0.15 # for BGL, lr
th = 0.99 # for BGL, if


for th in np.arange(0.01, 1, 0.05):
    # pred_train = (proba_train>th).astype(int)
    # precision, recall, f1, _ = precision_recall_fscore_support(train_y, pred_train, average='binary')
    # print("Train accuracy:")
    # print('Precision: {:.3f}, recall: {:.3f}, F1-measure: {:.3f}\n'.format(precision, recall, f1))
    pred_test = (proba_test>th).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(test_y, pred_test, average='binary')
    print("Test accuracy:")
    print('Precision: {:.3f}, recall: {:.3f}, F1-measure: {:.3f}\n'.format(precision, recall, f1))

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.912

Test accuracy:
Precision: 1.000, recall: 0.839, F1-measure: 0.

In [23]:
run demo\SVM_demo.py

====== Input data summary ======
Loading data/HDFS/HDFS_100k.log_structured.csv
struct_log
LineId                                                           1
Date                                                         81109
Time                                                        203518
Pid                                                            143
Level                                                         INFO
Component                                 dfs.DataNode$DataXceiver
Content          Receiving block blk_-1608999687919862906 src: ...
EventId                                                         E5
EventTemplate             Receiving block <*> src: /<*> dest: /<*>
----------
LineId                                                           2
Date                                                         81109
Time                                                        203518
Pid                                                             35
Level                      

Precision: 1.000, recall: 0.236, F1-measure: 0.381



Exception ignored in: <bound method tqdm.__del__ of <tqdm.auto.tqdm object at 0x000001D44545DA90>>
Traceback (most recent call last):
  File "c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\tqdm\std.py", line 1147, in __del__
    self.close()
  File "c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\tqdm\notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html